In [16]:
import random
import pandas as scan
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [17]:
dataIn = scan.read_csv("DataLatih.csv",';')
dataFinal = dataIn.values
dataIn.head()

,Suhu,Waktu,Kondisi Langit,Kelembapan,Terbang
0,Normal,Siang,Berawan,Normal,Tidak
1,Normal,Siang,Cerah,Normal,Tidak
2,Normal,Pagi,Hujan,Normal,Tidak
3,Normal,Sore,Rintik,Rendah,Ya
4,Normal,Malam,Berawan,Tinggi,Tidak


In [18]:
encoder = OneHotEncoder(sparse = False)
dataLatihEncoded = encoder.fit_transform(dataIn)
dataLatihEncoded = np.delete(dataLatihEncoded,14,1)
scan.DataFrame(dataLatihEncoded).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [19]:
def getIndividu(nIndividu):
    box = []
    for i in range(0,nIndividu):
        box.append(random.randint(0,1))
    return np.array(box)
# print(getIndividu(15))

In [20]:
def getPopulasi(nPopulasi):
    boxPop = []
    for i in range(0,nPopulasi):
        nIndividu = np.random.randint(1,10)*15
        boxPop.append(getIndividu(nIndividu))
    return boxPop
# populations = getPopulasi(10)
# print(populations[0])

In [21]:
#hitung Fitnes
def fitness(populations,dataLatihEncoded):
    fitValues = []
    for chromosome in populations:
        jml = 0
        getRules = np.split(chromosome, round(len(chromosome)/15))
#         print(len(chromosome))
        for i in dataLatihEncoded:
            splitSuhu = i[0:3]
            splitWaktu = i[3:7]
            splitCuaca = i[7:11]
            splitKelembapan = i[11:14]
            splitStatus = i[14]
            for rule in getRules:
                splitChroSuhu = rule[0:3]
                splitChroWaktu = rule[3:7]
                splitChroCuaca = rule[7:11]
                splitChroKelembapan = rule[11:14]
                splitChroStatus = rule[14]
                if np.any(np.logical_and(splitSuhu,splitChroSuhu)) and np.any(np.logical_and(splitWaktu,splitChroWaktu)) and np.any(np.logical_and(splitCuaca,splitChroCuaca)) and np.any(np.logical_and(splitKelembapan,splitChroKelembapan)):
                    if(splitStatus == splitChroStatus):
                        jml=jml+1
                        break
                else:
                    if(splitStatus != splitChroStatus):
                        jml=jml+1
                        break
        fit = jml/(len(dataLatihEncoded))*100
        fitValues.append(fit)
    return fitValues

In [22]:
# pop = [np.array([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1])]
# dataFit =fitness(populations, dataLatihEncoded)
# dataFit

In [23]:
# Parent Selection
def RouletteWheel(fit, populations):
    total = 0
    for i in range(len(fit)):
        total = total + fit[i]
        
    r = np.random.uniform(0,1)
    individu = 0
    i = 0
    while r>0:
        r = r - (fit[i]/total)
        individu = individu+1
        i = i+1
    return populations[individu-1]
# RouletteWheel(dataFit, populations)
# # populations,dataFit

In [24]:
#crossOver (Pindah Silang)
def crossOver(parent1, parent2, probCrossOver):
    #getVarians
    pointer1p1 = random.randint(0,len(parent1))
    pointer2p1 = random.randint(0,len(parent1))
    if pointer1p1 > pointer2p1:
        pointer1p1, pointer2p1 = pointer2p1, pointer1p1  

    p1 = pointer2p1 - pointer1p1
    gap = p1 % 15
    varian1 = []
    varian1.append(pointer1p1)
    varian1.append(pointer1p1+p1)
    
    varian2 = []
    varian2.append(pointer1p1)
    varian2.append(pointer1p1+gap)

    varian3 = []
    varian3.append(abs(p1-pointer2p1))
    varian3.append(pointer2p1)
    
    varian4 = []
    varian4.append(abs(gap-pointer2p1))
    varian4.append(pointer2p1)
    
    p2 = []
    p2.append(varian1)
    p2.append(varian2)
    p2.append(varian3)
    p2.append(varian4)

    getRandomVarP2 = random.randint(0,3)    
    
    #ini mindahin parent1 ke tab sementara
    tab1Temp = []
    if pointer1p1 < pointer2p1:
        for i in range(pointer1p1,pointer2p1):
            tab1Temp.append(parent1[pointer1p1])
    elif pointer2p1 < pointer1p1:
        for i in range(pointer2p1,pointer1p1):
            tab1Temp.append(parent1[pointer2p1])
        
    selectedVarian = p2[getRandomVarP2]
      
    if selectedVarian[1] > len(parent2):
        pointer3p2 = random.randint(0,len(parent2)-gap)
        varian5 = []
        varian5.append(pointer3p2)
        varian5.append(pointer3p2+gap)
        #(c,c+gap)
        selectedVarian = varian5
    
    tab2Temp = []
    if selectedVarian[0] < selectedVarian[1]:
        for i in range(selectedVarian[0],selectedVarian[1]):
            tab2Temp.append(parent2[i])
    elif selectedVarian[1] < selectedVarian[0]:
        for i in range(selectedVarian[1],selectedVarian[0]):
            tab2Temp.append(parent2[i])
    
    #CrossOver
#     probCrossOver = 1
    if probCrossOver >= np.random.random():
        parent1 = np.delete(parent1, range(pointer1p1,pointer2p1))
        parent1 = np.insert(parent1, pointer1p1, tab2Temp)
        parent2 = np.delete(parent2, range(selectedVarian[0],selectedVarian[1]))
        parent2 = np.insert(parent2, selectedVarian[0], tab1Temp)
    
    return parent1, parent2
# print(crossOver(RouletteWheel(fitness(data,dataFinal))))
# tes1 = data[RouletteWheel(fitness(data,dataFinal))]
# tes2 = data[RouletteWheel(fitness(data,dataFinal))]
# print(crossOver(tes1,tes2))

In [25]:
# a = np.zeros(135)
# b = np.arange(0,135)
# a,b = crossOver(a,b,1)
# print(a,b,sep='\n')
# len(a),len(b)

In [26]:
#mutasi
def probMutation(tabel1, probMut):
    for i in range(len(tabel1)):
        if probMut>=np.random.uniform(0,1):
            if tabel1[i] == 0:
                tabel1[i] == 1
            elif tabel1[i] == 1:
                tabel1[i] == 0
    return tabel1

In [27]:
def elitisme(fit,population):
    sortPop = sorted(zip(fit,population), key=lambda x:x[0], reverse=True)
    return [sortPop[0][1],sortPop[1][1]]

In [37]:
#main Program
nPopulasi = 25
nIndividu = 15
probCrossOver = 0.7
probMut = 0.5
mainPopulation = getPopulasi(nPopulasi)
maxGen = 100
for i in range(maxGen):
    mainFit = fitness(mainPopulation, dataLatihEncoded)
    newPopulation = elitisme(mainFit,mainPopulation)
    while len(newPopulation) < nPopulasi:
        p1 = RouletteWheel(mainFit, mainPopulation).copy()
        p2 = RouletteWheel(mainFit, mainPopulation).copy()
        newChild1, newChild2 = crossOver(p1, p2, probCrossOver)
        newChild1 = probMutation(p1, probMut)
        newChild2 = probMutation(p2, probMut)
        newPopulation.append(newChild1)
        newPopulation.append(newChild2)
#     clear_output(wait=True)
#     print("gen ",i)
#     print(*list(zip(mainPopulation, mainFit)),sep="\n")    
    mainPopulation = newPopulation
# #     print(*mainFit,sep="\n")
hasilfit = fitness(mainPopulation, dataLatihEncoded)

In [38]:
# print(hasilfit, mainPopulation)
#dataLatih
# f = open("test.txt", "a")
dataUji = scan.read_csv("dataUji.csv", ';').drop(columns="Terbang/Tidak")
dataUji
encoderUji = OneHotEncoder(sparse = False)
dataUjiEncoded = encoderUji.fit_transform(dataUji).astype(int)
scan.DataFrame(dataUjiEncoded).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0,0,1,0,0,1,0,1,0,0,0,0,1,0
1,0,1,0,0,0,1,0,0,0,1,0,0,0,1
2,1,0,0,0,1,0,0,0,0,1,0,0,0,1
3,0,0,1,0,0,1,0,0,0,1,0,1,0,0
4,0,1,0,1,0,0,0,0,0,1,0,0,1,0


In [39]:
def klasifikasiDataUji(data, kromosom):
    answer = []
    getRules = np.split(kromosom, round(len(kromosom)/15))
    x = 0
    for i in data:
        splitSuhu = i[0:3]
        splitWaktu = i[3:7]
        splitCuaca = i[7:11]
        splitKelembapan = i[11:14]
        for rule in getRules:
            splitChroSuhu = rule[0:3]
            splitChroWaktu = rule[3:7]
            splitChroCuaca = rule[7:11]
            splitChroKelembapan = rule[11:14]
            splitChroStatus = rule[14]
            if np.any(np.logical_and(splitSuhu,splitChroSuhu)) and np.any(np.logical_and(splitWaktu,splitChroWaktu)) and np.any(np.logical_and(splitCuaca,splitChroCuaca)) and np.any(np.logical_and(splitKelembapan,splitChroKelembapan)):
                if splitChroStatus == 1:
                    answer.append("ya")
                    break
                else:
                    answer.append("tidak")
                    break
            elif x == len(getRules)-1:
#                 print('tidak')
#                 print()
                answer.append("tidak")
                break
            x+=1
    return answer
hasil=klasifikasiDataUji(dataUjiEncoded,mainPopulation[0])
print(len(hasil))
# hasilfit
scan.DataFrame(hasil).to_csv("hasil.csv")
print(hasil)

20
['ya', 'tidak', 'tidak', 'tidak', 'tidak', 'tidak', 'tidak', 'tidak', 'tidak', 'tidak', 'tidak', 'tidak', 'tidak', 'tidak', 'tidak', 'tidak', 'tidak', 'tidak', 'tidak', 'tidak']
